In [16]:
from pathlib import Path
from typing import List, Tuple, Optional, Union
from pydantic import BaseModel
from enum import Enum
import os
from joblib import Parallel, delayed
from dataclasses import dataclass

import importlib
import modeling
importlib.reload(modeling)
import modeling.riscv_dv as riscv_dv
importlib.reload(modeling.riscv_dv)
import modeling.coverage_parser as coverage_parser
importlib.reload(modeling.coverage_parser)

<module 'modeling.coverage_parser' from '/nscratch/vighneshiyer/cov-proxy-model/modeling/coverage_parser.py'>

In [22]:
class Timeout:
    pass
@dataclass
class Pass:
    pass_cycles: int
@dataclass
class Fail:
    fail_cycles: Optional[int] # None = failed for unknown reason (e.g. bad syscall), int = failed due to timeout in given cycles
    
@dataclass
class TestData:
    config: riscv_dv.RiscvDvConfig
    cov_data: Optional[List[coverage_parser.cov_t]]
    spike_status: Optional[int]  # None = timeout, int value = number of lines of spike.log (roughly correlated with number of instructions executed)
    rtl_sim_status: Union[Pass, Fail] # pass: cycles, fail: timeout and cycles, fail: no reason
    test_id: int
    test_iteration: int
    
def read_last_line(file: Path) -> str:
    with open(file, 'rb') as f:
        try:  # catch OSError in case of a one line file 
            f.seek(-2, os.SEEK_END)
            while f.read(1) != b'\n':
                f.seek(-2, os.SEEK_CUR)
        except OSError:
            f.seek(0)
        last_line = f.readline().decode()
    return last_line

In [27]:
def parse_test(test_id_folder: Path) -> List[TestData]:
    # Figure out how many iterations were run for a given test id, look at the maximum number of file names
    n_iterations: int = int(max([x.name.split('.')[0] for x in test_id_folder.iterdir()])[0])
    test_data: List[TestData] = []
    print(f"Parsing test {test_id_folder}")
    for i in range(n_iterations):
        riscv_dv_config = riscv_dv.RiscvDvConfig.parse_raw((test_id_folder / f"{i}.json").read_text())
        
        cov_dat_file = (test_id_folder / f"{i}.cov.dat")
        if cov_dat_file.exists():
            cov_data = [coverage_parser.parse_line(line) for line in cov_dat_file.read_text().split('\n')[1:] if len(line) != 0]
        else:
            cov_data = None
        
        spike_status_str = (test_id_folder / f"{i}.spike.status").read_text()
        if spike_status_str == "TIMEOUT":
            spike_status = None
        else:
            spike_status = int(spike_status_str)
            
        sim_log_file = (test_id_folder / f"{i}.sim.log")
        sim_log_str = read_last_line(sim_log_file)
        if sim_log_str.startswith("*** PASSED ***"):
            rtl_sim_status = Pass(pass_cycles=int(sim_log_str.split(' ')[5]))
        elif sim_log_str.startswith("*** FAILED *** via trace_count"):
            rtl_sim_status = Fail(fail_cycles=int(sim_log_str.split(' ')[9]))
        else:
            rtl_sim_status = Fail(fail_cycles=None)
        test_data.append(TestData(config=riscv_dv_config, cov_data=cov_data, spike_status=spike_status, rtl_sim_status=rtl_sim_status, test_id=test_id_folder.name, test_iteration=i))
    return test_data

In [28]:
# Get coverage data and riscv-dv configs for each test
data = Parallel(n_jobs=24)(delayed(parse_test)(test_id_folder) for test_id_folder in (Path.cwd() / "configs").iterdir())
len(data)

100

In [32]:
data[0][1].config

RiscvDvConfig(plusarg_config={'num_of_tests': '1', 'enable_page_table_exception': '0', 'enable_interrupt': '0', 'enable_nested_interrupt': '0', 'enable_timer_irq': '0', 'num_of_sub_program': '5', 'instr_cnt': '200', 'no_ebreak': '1', 'no_ecall': '1', 'no_dret': '1', 'no_wfi': '1', 'no_branch_jump': '1', 'no_load_store': '1', 'no_csr_instr': '0', 'fix_sp': '1', 'use_push_data_section': '0', 'enable_illegal_csr_instruction': '0', 'enable_access_invalid_csr_level': '0', 'enable_misaligned_instr': '0', 'enable_dummy_csr_write': '0', 'allow_sfence_exception': '0', 'no_data_page': '1', 'no_directed_instr': '0', 'no_fence': '1', 'no_delegation': '1', 'illegal_instr_ratio': '63', 'hint_instr_ratio': '2', 'gen_all_csrs_by_default': '0', 'gen_csr_ro_write': '0', 'num_of_harts': '1', 'enable_unaligned_load_store': '0', 'force_m_delegation': '0', 'force_s_delegation': '0', 'require_signature_addr': '0', 'disable_compressed_instr': '0', 'randomize_csr': '0', 'tvec_alignment': '2', 'gen_debug_sectio